In [74]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/0r/tspq5j2s2fv0zp31f7l4m0_c0000gn/T/ipykernel_90185/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [75]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [76]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [77]:
# Your code
from sklearn.model_selection import train_test_split

train_data = data.copy()
test_data = pd.read_csv('data/kg_test.csv', encoding='latin-1', skip_blank_lines=True)

## Data Preprocessing

In [78]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andsto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [79]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [80]:
# Your code
import re

def clean_html(html : str) -> str:
    html_no_js = re.sub(r'<script[^>]*>.*?</script>', ' ', html, flags=re.DOTALL | re.IGNORECASE)
    html_no_inline_css = re.sub(r'style="[^"]*"', ' ', html_no_js, flags=re.IGNORECASE)
    html_no_comments = re.sub(r'<!--.*?-->', ' ', html_no_inline_css, flags=re.DOTALL | re.IGNORECASE)
    text_only = re.sub(r'<[^>]+>', ' ', html_no_comments)
    return text_only

train_data['text'] = train_data['text'].apply(clean_html)
test_data = test_data['text'].apply(clean_html)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [81]:
# Your code
def clean_text(text : str) -> str:
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]', ' ', text) # remove all special characters
    cleaned_text = re.sub(r'\b\w\b', ' ', cleaned_text) # remove single characters
    cleaned_text = re.sub(r'\d+', ' ', cleaned_text) # remove numbers
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip() # substitute white spaces
    cleaned_text = re.sub(r'^b\s+', '', cleaned_text) # remove prefixed 'b'
    return cleaned_text.lower()

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = train_data['text'].apply(clean_text)

In [82]:
# train_data['text']

## Now let's work on removing stopwords
Remove the stopwords.

In [83]:
# Your code
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andsto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
stopWords = set(stopwords.words('english'))
stopWords

def remStopWords(text):
    cleanWords = []
    words = text.split()
    for word in words:
        if word.lower() not in stopWords:
            cleanWords.append(word)
    return ' '.join(cleanWords)

train_data['text'] = train_data['text'].apply(remStopWords)
train_data

,text,label
0,dear sir strictly private business proposal mi...,1
1,,0
2,nora cheryl emailed dozens memos haiti weekend...,0
3,dear sir fmadam c know proposal might surprise...,1
4,fyi,0
...,...,...
995,latest sounds contradictory af decide shall ta...,0
996,transfer million pounds youraccount name mr ej...,1
997,barb call explain back country,0
998,yang travelnot free tonite may work tomorrow,0


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [85]:
# Your code

from nltk.corpus import wordnet

wl = nltk.WordNetLemmatizer()

# nltk.download('corpora/wordnet')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0]#.upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)

def lemmatize_text(text : str) -> str:
    word_list = []
    for word in text.split():
        word_list.append(wl.lemmatize(word, get_wordnet_pos(word)))
    return (' '.join(map(str, word_list)))

train_data['text'] = train_data['text'].apply(lemmatize_text)
train_data


,text,label
0,dear sir strictly private business proposal mi...,1
1,,0
2,nora cheryl email dozen memo haiti weekend ple...,0
3,dear sir fmadam c know proposal might surprise...,1
4,fyi,0
...,...,...
995,late sound contradictory af decide shall take ...,0
996,transfer million pound youraccount name mr eji...,1
997,barb call explain back country,0
998,yang travelnot free tonite may work tomorrow,0


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [86]:
# Your code

from nltk.tokenize import word_tokenize
bag = train_data['text'].copy()
bag.apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)
bag

0      dear sir strictly private business proposal mi...
1                                                       
2      nora cheryl email dozen memo haiti weekend ple...
3      dear sir fmadam c know proposal might surprise...
4                                                    fyi
                             ...                        
995    late sound contradictory af decide shall take ...
996    transfer million pound youraccount name mr eji...
997                       barb call explain back country
998         yang travelnot free tonite may work tomorrow
999    sbwhoeopsunday february pmhshaunh talk shaun w...
Name: text, Length: 1000, dtype: object

In [87]:
from collections import Counter
words = re.findall(r'\b\w{3,}\b', ' '.join(bag)) # we only count words with 3 or more characters
word_counts = Counter(words)
top10 = word_counts.most_common(10)
top10

[('money', 993),
 ('account', 906),
 ('bank', 811),
 ('fund', 789),
 ('transfer', 581),
 ('country', 565),
 ('transaction', 555),
 ('business', 541),
 ('nbsp', 475),
 ('million', 474)]

## Extra features

In [73]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

KeyError: 'preprocessed_text'

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code